### Imports

In [2]:
import pandas as pd
import numpy as np

#### Source

http://robotics.ethz.ch/~asl-datasets/ijrr_euroc_mav_dataset/machine_hall/


#### Background

The flight environment (an indoor factory hall)
Easy → the level of difficulty of the flight path

#### Data

IMU_data/data.csv → IMU data (what the drone measured)

This is Analogous to kite’s onboard sensors.

Sampling rate = 200 Hz -> dt =  0.005 s.



state_groundtruth_data/data.csv (Ground-truth data) measured with A motion-capture system providing ground-truth position

-> contains (v_x, v_y, v_z) linear velocity (m/s) which is target

Sampling rate = 100 Hz -> dt =  0.01 s.

#### Task description

Will use RNN to predict velocity (vx,vy,vz) from IMU data.

#### Data handling

Currently, the IMU measurements and the groundtruth have different timestamps dt = 0.005 for IMU and dt = 0.01 for groundtruth -> Resample both to dt = 0.02.

In [3]:
import torch

IMU = pd.read_csv("../Data/IMU_data/data.csv")
groundtruth = pd.read_csv("../Data/state_groundtruth_data/data.csv")

IMU.drop(index = 0, inplace = True) #Because of different timestamps

"""Adjust such that timestamp start at 0 and so that time-spaces are even in Input and output"""

IMU['t'] = (IMU['#timestamp [ns]'] - IMU['#timestamp [ns]'].iloc[0]) * 1e-9
groundtruth['t'] = (groundtruth['#timestamp'] - groundtruth['#timestamp'].iloc[0]) * 1e-9
IMU = IMU.iloc[:len(groundtruth)]

"""Write to tensors"""

IMU.drop(columns=['#timestamp [ns]','t'], inplace=True)
X_np = IMU.to_numpy()

groundtruth_x_vel = groundtruth[[' v_RS_R_x [m s^-1]']]
Y_np = groundtruth_x_vel.to_numpy()

X = torch.from_numpy(X_np).float()
Y = torch.from_numpy(Y_np).float()

print(X)

print(len(X))
seq_len = 5
for t in range(seq_len):
    print(t)

tensor([[-0.0991,  0.1403,  0.0293,  8.0333, -0.4086, -2.4026],
        [-0.0984,  0.1278,  0.0377,  7.8862, -0.4250, -2.4353],
        [-0.1026,  0.1159,  0.0454,  7.8290, -0.3759, -2.4680],
        ...,
        [ 0.0426,  0.0119,  0.3114,  8.5972,  0.4250, -3.1953],
        [ 0.0600,  0.1564,  0.3211, 10.3624, -0.2125, -3.4568],
        [ 0.0565,  0.0489,  0.3260,  9.6596, -0.0327, -3.1381]])
36382
0
1
2
3
4
